In [1]:
import psycopg2
from psycopg2 import Error
from psycopg2.extras import NamedTupleCursor
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

In [2]:
def create_connection(db_name, db_user, db_password, db_host, db_port):
    connection = None
    try:
        connection = psycopg2.connect(
            database=db_name,
            user=db_user,
            password=db_password,
            host=db_host,
            port=db_port
        )
        print("Connection to PostgreSQL DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")
    return connection

def execute_query(connection, query, fetch_result=False):
    cursor = connection.cursor(cursor_factory=NamedTupleCursor)
    cursor.execute(query)
    connection.commit()
    if fetch_result:
        return cursor.fetchall()


def execute_queries(db, sql_commands):
    db.autocommit = True
    with db.cursor() as cursor:
        for sql_command in sql_commands:
            cursor.execute(sql_command)

In [3]:
db = create_connection("shopdb", "postgres", "94Q2%WRJ61", "localhost", "5432")

Connection to PostgreSQL DB successful


In [4]:
creating_tables = [
"DROP TABLE IF EXISTS instrument;",
"DROP TABLE IF EXISTS status;",
"DROP TABLE IF EXISTS location;",
"DROP TABLE IF EXISTS material;",
"DROP TABLE IF EXISTS guitar;",
"DROP TABLE IF EXISTS keyboard;",
"DROP TABLE IF EXISTS drum;",
"DROP TABLE IF EXISTS shape;",
"DROP TABLE IF EXISTS manufacturer;",
"DROP TABLE IF EXISTS employees;",
"DROP TABLE IF EXISTS order_history;",
"DROP TABLE IF EXISTS orders_instruments;",
"""CREATE TABLE instrument(
"id" SERIAL PRIMARY KEY,
"instr_type" INT NOT NULL,
"manufacturer" INT NOT NULL,
"material" INT NOT NULL,
"location" INT NOT NULL,
"status" INT NOT NULL,
"price" INT NOT NULL
);
""",
"""CREATE TABLE status(
	"status_id" INT NOT NULL,
	"status_name" TEXT NOT NULL
    );
""",
"""CREATE TABLE location(
	"location_id" INT NOT NULL,
	"address" TEXT NOT NULL
    );
""",
"""CREATE TABLE material(
	"material_id" INT NOT NULL,
	"material_name" TEXT NOT NULL
    );
""",
"""CREATE TABLE guitar(
	"id" INT NOT NULL,
	"string_num" INT NOT NULL,
	"fret_num" INT NOT NULL,
	"floyd_rose" boolean NOT NULL,
	"shape" INT NOT NULL
    );
""",
"""CREATE TABLE shape(
	"shape_id" INT NOT NULL,
	"shape_name" TEXT NOT NULL
    );
""",
"""CREATE TABLE keyboard(
	"id" INT NOT NULL,
	"keynum" INT NOT NULL,
	"is_synth" boolean NOT NULL
    );
""",
"""CREATE TABLE drum(
	"id" INT NOT NULL,
	"diameter" INT NOT NULL
    );
""",
"""CREATE TABLE employees(
	"employee_id" INT NOT NULL,
	"employee_name" TEXT NOT NULL
    );
""",
"""CREATE TABLE order_history(
	"order_id" INT NOT NULL,
	"date" DATE NOT NULL,
	"location_id" INT NOT NULL,
	"cashier_id" INT NOT NULL
    );
""",
"""CREATE TABLE manufacturer(
	"manufacturer_id" INT NOT NULL,
	"manufacturer_name" VARCHAR(255) NOT NULL
    );
""",
"""CREATE TABLE orders_instruments(
	"order_id" BIGINT NOT NULL,
	"instrument_id" INT NOT NULL
    );
"""]
execute_queries(db, creating_tables)

In [5]:
filling_static_tables = [
    """ INSERT INTO public.location (location_id, address)
    VALUES
        (1, 'Moscow'),
        (2, 'St. Petersburg'),
        (3, 'Kazan')
        """,
    """INSERT INTO public.status (status_id, status_name)
    VALUES
        (1, 'in stock'),
        (2, 'in shop'),
        (3, 'booked')
        """,
    """INSERT INTO public.material (material_id, material_name)
        VALUES
            (1, 'plastic'),
            (2, 'wood'),
            (3, 'carbon'),
            (4, 'metal')
            """,
    """INSERT INTO public.manufacturer (manufacturer_id, manufacturer_name)
        VALUES
            (1, 'Gibson'),
            (2, 'Yamaha'),
            (3, 'Fender'),
            (4, 'Remo')
            """,
    """INSERT INTO public.shape (shape_id, shape_name)
        VALUES
            (1, 'Telecaster'),
            (2, 'Stratocaster'),
            (3, 'Explorer'),
            (4, 'Flying V')
            """,
    """INSERT INTO public.employees (employee_id, employee_name)
        VALUES
            (1, 'Mark'),
            (2, 'Egor'),
            (3, 'Irina')
            """]
execute_queries(db, filling_static_tables)

In [6]:
filling_dynamic_tables = [
    """INSERT INTO public.instrument (instr_type, manufacturer, material, location, status, price)
        VALUES
            (1, 1, 2, 1, 1, 10000),
            (1, 2, 2, 2, 1, 15000),
            (1, 2, 2, 2, 2, 15500),
            (1, 2, 2, 3, 1, 23000),
            (1, 1, 2, 1, 1, 50000),
            (2, 3, 1, 1, 1, 6000),
            (2, 3, 2, 2, 2, 80000),
            (2, 3, 1, 1, 1, 11000),
            (2, 2, 1, 3, 2, 12990),
            (3, 4, 2, 3, 2, 8000),
            (3, 4, 3, 1, 2, 4000),
            (3, 4, 3, 1, 2, 4500)
    """,
    """INSERT INTO guitar (id, string_num, fret_num, floyd_rose, shape)
        VALUES
            (1, 6, 21, false, 1),
            (2, 6, 21, false, 2),
            (3, 6, 22, true, 1),
            (4, 6, 21, true, 3),
            (5, 7, 22, false, 2)
    """,
    """INSERT INTO keyboard (id, keynum, is_synth)
        VALUES
            (6, 52, true),
            (7, 108, false),
            (8, 64, true),
            (9, 64, false)
    """,
    """INSERT INTO drum (id, diameter)
            VALUES
                (10, 32),
                (11, 30),
                (12, 30)
        """,
    """INSERT INTO order_history (order_id, date, location_id, cashier_id)
            VALUES
                (1, '2022-12-01', 1, 1),
                (2, '2022-12-02', 1, 2),
                (3, '2022-12-02', 2, 3),
                (4, '2022-12-08', 1, 2)
        """,
    """INSERT INTO orders_instruments (order_id, instrument_id)
            VALUES
                (1, 2),
                (1, 4),
                (2, 5),
                (3, 11),
                (3, 12),
                (4, 8),
                (4, 9)
        """]
execute_queries(db, filling_dynamic_tables)

Шаблон кода запроса:

request_X = """ """
result = execute_query(db, request_X, fetch_result=True)
for row, value in enumerate(result):
    print(row, value)

In [7]:
request_1 = """SELECT id, floyd_rose, string_num 
FROM guitar JOIN shape ON guitar.shape = shape.shape_id
WHERE shape_name = 'Explorer'"""
result = execute_query(db, request_1, fetch_result=True)
for row, value in enumerate(result):
    print(row, value)

0 Record(id=4, floyd_rose=True, string_num=6)


In [8]:
request_2 = """SELECT manufacturer_name, price
FROM instrument i JOIN drum ON i.id = drum.id JOIN manufacturer ON i.manufacturer = manufacturer_id
WHERE price > 4000 AND price < 5000"""
result = execute_query(db, request_2, fetch_result=True)
for row, value in enumerate(result):
    print(row, value)

0 Record(manufacturer_name='Remo', price=4500)


In [9]:
request_3 = """SELECT address FROM location
WHERE address NOT LIKE '% %'"""
result = execute_query(db, request_3, fetch_result=True)
for row, value in enumerate(result):
    print(row, value)

0 Record(address='Moscow')
1 Record(address='Kazan')


In [10]:
request_4 = """SELECT (
(SELECT count (*) FROM keyboard WHERE is_synth = True)
/
(SELECT count (is_synth) FROM keyboard WHERE is_synth = False))"""
result = execute_query(db, request_4, fetch_result=True)
for row, value in enumerate(result):
    print(row, value)

0 Record(f_column_=1)


In [11]:
request_5 = """SELECT order_history.order_id, sum(price)
FROM order_history
INNER JOIN orders_instruments ON order_history.order_id = orders_instruments.order_id
INNER JOIN instrument ON orders_instruments.instrument_id = instrument.id
WHERE date > '2022-12-01'
GROUP BY order_history.order_id, instr_type""" 
# запрос на даты; выводит ID заказов, поступивших после 1 декабря, и суммы цен инструментов в этом заказе
result = execute_query(db, request_5, fetch_result=True)
for row, value in enumerate(result):
    print(row, value)

0 Record(order_id=3, sum=8500)
1 Record(order_id=2, sum=80000)
2 Record(order_id=4, sum=23990)


In [12]:
request_6 = """select count(*), instr_type FROM instrument where instr_type = 1
group by instr_type
union
select count (*), instr_type FROM instrument where instr_type = 2
group by instr_type
union
select count (*), instr_type FROM instrument where instr_type = 3
group by instr_type
order by count DESC
LIMIT 1""" 
# выводит количество инструментов самого многочисленного типа и его ID 
# (1-гитары, 2-клавишные, 3-барабаны)
result = execute_query(db, request_6, fetch_result=True)
for row, value in enumerate(result):
    print(row, value)

0 Record(count=5, instr_type=1)


In [27]:
request_7 = """SELECT shape.shape_name, count(shape.shape_name)
FROM order_history 
JOIN orders_instruments ON order_history.order_id = orders_instruments.order_id
JOIN instrument ON orders_instruments.instrument_id = instrument.id
JOIN guitar ON guitar.id = instrument.id
JOIN shape ON shape.shape_id = guitar.shape
WHERE date >= '2022-12-01'
GROUP BY shape_name"""
# сколько гитар какой формы купили, начиная с 1 декабря
# ответ: 1 стратокастер, 1 эксплорер
result = execute_query(db, request_7, fetch_result=True)
for row, value in enumerate(result):
    print(row, value)

0 Record(shape_name='Stratocaster', count=1)
1 Record(shape_name='Explorer', count=1)


In [41]:
# топ 3 кассира по сумме проданных товаров --- НЕ ЗАКОНЧЕНО
request_8 = """SELECT employee_name, sum(price)
FROM employees 
JOIN order_history ON employee_id = cashier_id
JOIN orders_instruments ON order_history.order_id = orders_instruments.order_id
JOIN instrument ON orders_instruments.instrument_id = instrument.id
GROUP BY employee_name, price
ORDER BY price DESC LIMIT 3
"""
result = execute_query(db, request_8, fetch_result=True)
for row, value in enumerate(result):
    print(row, value)

0 Record(employee_name='Egor', sum=80000)
1 Record(employee_name='Mark', sum=23000)
2 Record(employee_name='Mark', sum=15000)
